In [2]:
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer
from threading import Thread
import numpy as np

def test_original_model():
    model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

    # 원본 모델과 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True
    )
    model.eval()

    PROMPT = '''You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.'''
    instructions = [
        "서울의 유명한 관광 코스를 만들어줄래?",
        "한국 전통 음식 중 외국인에게 추천할 만한 것은?",
        "기후 변화에 대응하기 위해 개인이 할 수 있는 일은?"
    ]

    results = []
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    for instruction in instructions:
        messages = [
            {"role": "system", "content": PROMPT},
            {"role": "user", "content": instruction}
        ]
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

        start_time = time.time()
        ttft = None
        generated_text = ""

        def generation():
            with torch.no_grad():
                model.generate(
                    input_ids,
                    max_new_tokens=2048,
                    eos_token_id=terminators,
                    do_sample=True,
                    temperature=0.6,
                    top_p=0.9,
                    repetition_penalty=1.1,
                    no_repeat_ngram_size=3,
                    streamer=streamer
                )

        thread = Thread(target=generation)
        thread.start()

        for new_text in streamer:
            if ttft is None:
                ttft = time.time() - start_time
            generated_text += new_text

        thread.join()
        end_time = time.time()

        inference_time = end_time - start_time
        token_count = len(tokenizer.encode(generated_text))
        tokens_per_second = token_count / inference_time

        result = {
            "instruction": instruction,
            "generated_text": generated_text,
            "inference_time": inference_time,
            "token_count": token_count,
            "tokens_per_second": tokens_per_second,
            "ttft": ttft
        }
        results.append(result)

    # 모델 크기 계산 (GB 단위)
    # model_size = sum(p.numel() * p.element_size() for p in model.parameters()) / (1024**3)

    return results

# 테스트 실행
results = test_original_model()

print(f"{'='*50}")
print("Original Model Evaluation Results")
print(f"{'='*50}")

# print(f"Model Size: {model_size:.2f} GB")

avg_inference_time = np.mean([r["inference_time"] for r in results])
avg_tokens_per_second = np.mean([r["tokens_per_second"] for r in results])
avg_ttft = np.mean([r["ttft"] for r in results])

print(f"Average Inference Time: {avg_inference_time:.2f} seconds")
print(f"Average Tokens/Second: {avg_tokens_per_second:.2f}")
print(f"Average Time to First Token: {avg_ttft:.4f} seconds")

print("\nFull Outputs:")
for idx, result in enumerate(results, 1):
    print(f"\n{'-'*40}")
    print(f"Sample {idx}:")
    print(f"Instruction: {result['instruction']}")
    print(f"Generated Text:\n{result['generated_text']}")
    print(f"Inference Time: {result['inference_time']:.2f} seconds")
    print(f"Token Count: {result['token_count']}")
    print(f"Tokens/Second: {result['tokens_per_second']:.2f}")
    print(f"Time to First Token: {result['ttft']:.4f} seconds")

print("\nTest completed.")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Original Model Evaluation Results
Average Inference Time: 26.01 seconds
Average Tokens/Second: 23.65
Average Time to First Token: 1.8135 seconds

Full Outputs:

----------------------------------------
Sample 1:
Instruction: 서울의 유명한 관광 코스를 만들어줄래?
Generated Text:
물론이죠! 서울은 다양한 역사적, 문화적 명소와 현대적인 아케이드가 공존하는 도시로, 많은 관광객들이 찾는 인기 있는 여행지입니다. 아래는 서울에서 방문해야 할 주요 관광지를 포함한 일일 관광코스입니다.

### 서울 일일관광코스로 가는 길

#### 1단계: 아침
- **경복궁**: 한국의 대표적인 조선 왕조의 궁궐로, 아침 일찍 방문하면 더 조용하고 멋진 사진을 찍을 수 있습니다.
  - 주소: 서울특별시 종로구 경복궁로 161
  - 시간: 오전 9:00 ~ 오후 5:00 (월요일: 휴장)

#### 2단계 중간 식사
- 근처에는 다양한 전통 한식당이나 카페가 많습니다. 경복원 내 또는 인근에서 간단히 점심을 즐겨보세요.

#### 3단계:
- **북촌 한옥마을**: 전통한옥이 모여있는 고즈넉한 마을로, 사진 촬영하기 좋은 곳입니다.
  주소 : 서울특별시 종로는 북촌 한오클라스트리 거리 및 그 주변

####4단계
- 기념품 쇼핑
  + **인사동**: 전統 한복, 기념물품 등을 구입할 수 있는 인기가 높은 지역입니다.
    주소 서울특별시 종로서길

####5단계 저녁 식사 및 휴식
- 인사동 인근에 위치한 레스토랑에서 저녁을 즐기거나, 인사 동거리에서 전통 예술 공연을 감상해보세요.

### 옵션 코스
- 서울타워: 저녁 시간대에 서울타운을 방문하여 야경을 감상을 하고, 전망대에서 서울의 밤 풍경을 볼 수도 있습니다.
    - 주소를 서울특별시 동작구 상도동 105-1

### 추가 